In [1]:
import pandas as pd
import seaborn as sns
sns.set_theme()

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (13,9)

vacs = pd.read_csv("./data/covid19-france-stock-vaccin-departement.csv", 
                   names = ['dept_code','dept_name','type','nb_dose','z1','date','z2','z3','iso3166-3','geo_point'], 
                   skiprows=1,
                   sep=';')
vacs = vacs.drop(labels=['z1','z2','z3'], axis=1)

# type conversions
vacs.date = pd.to_datetime(vacs.date, format="%Y-%m-%d")
vacs.type = pd.Categorical(vacs.type)

# date to run analysis on, few dates per month only
analysis_date = '2021-07-29'
vacs = vacs[vacs.date==analysis_date]

vacs.dtypes
vacs.tail()

,dept_code,dept_name,type,nb_dose,date,iso3166-3,geo_point
7682,64,Pyrénées-Atlantiques,Moderna,570,2021-07-29,FXX,"43.2562009633,-0.760761078137"
7683,86,Vienne,Pfizer,22188,2021-07-29,FXX,"46.5652473368,0.459135535564"
7684,90,Territoire de Belfort,Pfizer,19782,2021-07-29,FXX,"47.6315421523,6.92857662641"
7685,93,Seine-Saint-Denis,Moderna,13580,2021-07-29,FXX,"48.9177348857,2.47790682097"
7686,93,Seine-Saint-Denis,AstraZeneca,2250,2021-07-29,FXX,"48.9177348857,2.47790682097"


In [2]:
dose_per_dept = vacs.groupby(['dept_code', 'dept_name'], as_index=False)['nb_dose'].sum()
dose_per_dept.nlargest(9, 'nb_dose')

,dept_code,dept_name,nb_dose
59,59,Nord,120253
74,75,Paris,113441
61,62,Pas-de-Calais,112873
82,83,Var,104840
68,69,Rhône,98453
12,13,Bouches-du-Rhône,97760
5,06,Alpes-Maritimes,96452
93,971,Guadeloupe,93496
44,44,Loire-Atlantique,91824


In [5]:
import geopandas
base = geopandas.read_file('./data/departements-20180101.shp')
base.head()

,geometry
0,"MULTIPOLYGON (((55.21643 -21.03904, 55.21652 -..."
1,"POLYGON ((1.68872 43.27368, 1.69001 43.27423, ..."
2,"POLYGON ((3.08206 45.28988, 3.08209 45.29031, ..."
3,"MULTIPOLYGON (((4.23014 43.46047, 4.23025 43.4..."
4,"POLYGON ((-0.14058 44.22648, -0.12931 44.23218..."


In [4]:
# filter out three-letter dpt codes (outre-mer?) 
base = base[base.code_insee.str.len() <= 2]

AttributeError: 'GeoDataFrame' object has no attribute 'code_insee'

In [ ]:
mg = pd.merge(base, dose_per_dept, left_on='code_insee', right_on='dept_code')
ax = mg.plot(column='nb_dose', cmap='YlGn', legend=True)
ax.set_title('Vaccin stock per dpt - ' + analysis_date)

In [ ]:
# df top9 departments with most vaccines
top9 = vacs[vacs.dept_code.isin(dose_per_dept.nlargest(9, 'nb_dose').dept_code)]

In [ ]:
# Initialize grid and map barplots
grid = sns.FacetGrid(top9, col='dept_name', hue='type', palette='pastel',
                     col_order = dose_per_dept.nlargest(9, 'nb_dose').dept_name,
                     col_wrap=3, height=4)
vac_list = ['Pfizer','AstraZeneca','Moderna','Janssen']
grid.map(sns.barplot,'type','nb_dose', order=vac_list)
grid.set_titles(col_template="{col_name}", row_template="{row_name}")
grid.add_legend(label_order=vac_list)